In [1]:
import sys
sys.path.append('../')
from rocky.models.LogLinear import LogLinear
from rocky.triangle import Triangle

In [2]:
t = Triangle.from_taylor_ashe()
t.df

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
ll = LogLinear(id='loss', tri=t)
ll

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.


loglinear()

In [4]:
ll.X_train.head()

,intercept,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_002,development_period_003,development_period_004,development_period_005,development_period_006,development_period_007,development_period_008,development_period_009,development_period_010
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
ll.X_forecast.head()

,intercept,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_002,development_period_003,development_period_004,development_period_005,development_period_006,development_period_007,development_period_008,development_period_009,development_period_010
19,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
28,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0
29,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
37,1,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0
38,1,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0


In [6]:
ll.y_train[:5]

0    357848.0
1    352118.0
2    290507.0
3    310608.0
4    443160.0
Name: y, dtype: float64

In [9]:
ll.GetY('train', log=False)[:5]

0    357848.0
1    352118.0
2    290507.0
3    310608.0
4    443160.0
Name: y, dtype: float64